# 训练代码

1. 保存模型和数据集
2. 处理数据集
3. 开启ms-swift训练

---
1. 保存模型和数据集

In [ ]:
!modelscope download --model Qwen/Qwen3-0.6B  --local_dir /data/nvme1/weights/Qwen3-0.6B
!modelscope download --dataset liucong/Chinese-DeepSeek-R1-Distill-data-110k-SFT  --local_dir /data/nvme0/zh_cot_110k_sft

Processing 4 items:   0%|                                 | 0/4 [00:00<?, ?it/s]






Processing 4 items:  25%|██████▎                  | 1/4 [00:00<00:01,  1.84it/s]
























































































































































































































































































































































































































































































Processing 4 items: 100%|█████████████████████████| 4/4 [00:32<00:00,  8.06s/it]


In [2]:
from modelscope.msdatasets import MsDataset
ds =  MsDataset.load('/data/nvme0/zh_cot_110k_sft', subset_name='default', split='train')
ds[0]

/home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 110000 examples [00:01, 96150.44 examples/s]


{'instruction': '能给我讲一个寓意深刻的故事吗？',
 'input': '',
 'output': '<think>\n好的，用户让我讲一个寓意深刻的故事。首先，我需要确定用户的需求是什么。他们可能想要一个能引发思考、有深层含义的故事，而不仅仅是娱乐性的。接下来，我得考虑故事的题材。寓言、现代故事还是神话改编？用户没有指定，所以可能需要一个通用且容易理解的类型。\n\n然后，主题方面，用户要求寓意深刻，所以需要选择一个普遍的主题，比如生命的意义、自我认知、成长等。考虑到现代人可能面临的困惑，比如存在的意义、工作的重复性，这样的主题更容易引起共鸣。\n\n接下来是角色设定。主角需要 relatable，可能是一个普通人，或者象征性的角色，比如机器人，这样更容易带入寓意。机器人有程序设定，却寻求自由，这样的冲突明显，容易展开。\n\n故事结构方面，可能需要一个循环的结构，来强调主题。比如开始和结束在同一个地方，但主角有了新的认识。这样能突出转变和领悟。\n\n然后考虑情节发展。主角觉醒，逃离，经历外面的世界，发现真相，最终回归。这样的结构可以展示变化的过程，并带出寓意。中间的导师角色可以帮助传递主题，比如老园丁的话，暗示生命的意义在于过程而非目的。\n\n在细节描写上，需要加入象征元素，比如编号、机械的工作、自然界的对比。老园丁的沙漏、胡须里的嫩芽，这些意象能增强故事的深度和隐喻。\n\n最后，结尾部分要让主角领悟到真谛，即使回到原地，但心态不同，带出主题：存在的意义在于体验本身。这样用户能得到启示，符合寓意深刻的要求。\n\n需要检查故事是否流畅，有没有逻辑漏洞，以及寓意是否明确。可能还需要调整语言，使其更生动，有画面感，让读者更容易沉浸其中。确保故事不长不短，结构紧凑，结尾点题，让用户回味。\n\n</think>\n\n\n\n好的，我将按照您的要求创作一个富有寓意的故事。为了让故事更完整，我会先列出一些基本的情节脉络。请您看看以下内容是否符合您的预期。如果您有其他想法，可以随时提出，我会进行调整。\n\n### 世界观设定\n\n- **时空背景**：近未来时空管理局内部，具有永恒循环属性的微观宇宙观测站。\n\n- **核心矛盾**：觉醒的机械观测员发现世界真相后，在永恒职责与自由意志之间的挣扎。\n\n- **特殊设定**：宇宙沙漏中的星

---
2. 处理数据集

In [3]:
!python ./ms-swift-train/alpaca2swift_dataset.py

创建目录：./data
训练数据已成功保存到 ./data/train.jsonl，共 5000 条
评估数据已成功保存到 ./data/eval.jsonl，共 500 条


---
3. 开启ms-swift训练
 
  - 单卡
  - 单机多卡分布式

In [14]:
!bash ms-swift-train/train_gpu0.sh

run sh: `/home/finetune/miniconda3/envs/eval/bin/python3.10 /home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/swift/cli/sft.py --model /data/nvme1/weights/Qwen3-0.6B --train_type lora --dataset ./data/train.jsonl --val_dataset ./data/eval.jsonl --torch_dtype bfloat16 --num_train_epochs 2 --per_device_train_batch_size 2 --per_device_eval_batch_size 2 --learning_rate 1e-4 --lora_rank 8 --lora_alpha 32 --target_modules all-linear --gradient_accumulation_steps 8 --eval_steps 50 --save_steps 50 --save_total_limit 2 --logging_steps 5 --max_length 2048 --output_dir /data/nvme1/weights/Qwen3_sft_eval --warmup_ratio 0.05 --dataloader_num_workers 4 --model_author swift --model_name qwen3 --report_to swanlab --swanlab_project swift-qwen3`
[INFO:swift] Successfully registered `/home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift] Loading the mo

In [17]:
!bash ms-swift-train/train_deepspeed.sh

run sh: `/home/finetune/miniconda3/envs/eval/bin/python3.10 -m torch.distributed.run --nproc_per_node 4 /home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/swift/cli/sft.py --model /data/nvme1/weights/Qwen3-0.6B --train_type lora --dataset ./data/train.jsonl --val_dataset ./data/eval.jsonl --torch_dtype bfloat16 --num_train_epochs 1 --per_device_train_batch_size 1 --per_device_eval_batch_size 1 --learning_rate 1e-4 --lora_rank 8 --lora_alpha 32 --target_modules all-linear --gradient_accumulation_steps 4 --eval_steps 50 --save_steps 50 --save_total_limit 2 --logging_steps 5 --max_length 2048 --output_dir /data/nvme1/weights/Qwen3_sft_eval --warmup_ratio 0.05 --dataloader_num_workers 4 --model_author swift --model_name swift-qwen3 --report_to swanlab --swanlab_project swift-qwen3 --deepspeed zero2`

*****************************************
Setting OMP_NUM_THREADS environment variable for each process to be 1 in default, to avoid your system being overloaded, please further 

In [4]:
!bash evalscope-eval/merge.sh

run sh: `/home/finetune/miniconda3/envs/eval/bin/python3.10 /home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/swift/cli/export.py --adapters /data/nvme1/weights/Qwen3_sft_eval/v5-20250603-151411/checkpoint-604 --model /data/nvme1/weights/Qwen3-0.6B --output_dir /data/nvme1/weights/Qwen3_sft_eval/output --merge_lora true`
[INFO:swift] Successfully registered `/home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/swift/llm/dataset/data/dataset_info.json`.
[INFO:swift] Loading the model using model_dir: /data/nvme1/weights/Qwen3_sft_eval/v5-20250603-151411/checkpoint-604
[INFO:swift] Successfully loaded /data/nvme1/weights/Qwen3_sft_eval/v5-20250603-151411/checkpoint-604/args.json.
[INFO:swift] rank: -1, local_rank: -1, world_size: 1, local_world_size: 1
[INFO:swift] Loading the model using model_dir: /data/nvme1/weights/Qwen3-0.6B
Traceback (most recent call last):
  File "/home/finetune/miniconda3/envs/eval/lib/python3.10/site-packages/swift/cli/export.py", lin

# 评估代码

1. 基本使用
2. 模型API服务评测
3. 模型推理性能压测

---
## 1. 基本使用

因为本地磁盘空间不够，因此我会将所有的模型或者数据集存放到别处，如果本地磁盘空间足够，可以直接运行下面的代码查看

**需要注意Qwen3可以开启或者关闭思考模式，因此需要额外添加该参数的设置：**

--generation-config '{"max_new_tokens":2048,"chat_template_kwargs":{"enable_thinking": false}}' \

In [6]:
!bash ./evalscope-eval/cli/easy.sh

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


2025-06-04 12:48:59,763 - evalscope - INFO - Args: Task config is provided with CommandLine type.
2025-06-04 12:49:01,582 - evalscope - INFO - Loading model /data/nvme1/weights/Qwen3_sft_eval/output ...
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
ANTLR runtime and generated code versions disagree: 4.9.3!=4.7.2
2025-06-04 12:49:03,132 - evalscope - WARNING - Got local model dir: /data/nvme1/weights/Qwen3_sft_eval/output
2025-06-04 12:49:03,132 - evalscope - INFO - Updating generation config ...
2025-06-04 12:49:03,230 - evalscope - INFO - Dump task config to ./outputs/20250604_124859/configs/task_config_e86197.yaml
2025-06-04 12:49:03,232 - evalscope - INFO - {
    "model": "/data/nvme1/weights/Qwen3_sft_eval/output",
    "model_id": "output",
    "model_args": {
        "revision": "master",
        "precision": "torch.float16"
    },
  

如果本地磁盘空间不足，使用下面的命令行把数据集下载到本地磁盘

In [ ]:
!wget https://modelscope.oss-cn-beijing.aliyuncs.com/open_data/benchmark/data.zip
!unzip data.zip

然后把dataset-args参数中local_path修改成自己的保存数据集地址，然后运行下面的代码。

**需要注意的是，gsm8k的数据集评测时默认的prompt_template为Question: {query}\nLet's think step by step\nAnswer:。测试过后发现enable_thinking设置为false比true要高些，所以可以默认为false，但是其他的没有cot提示的测试集可以设置为true，尤其是数学推理的时候**

In [ ]:
!bash evalscope-eval/cli/gsm8k.sh
!bash evalscope-eval/cli/multi.sh

---
## 2、模型API服务评测

这里适合于模型结构没有被evalscope集成或者有自定义评测标准使用，其实这一步也很简单，只需要下面三个步骤思路：

1. **接收用户请求**：用户通过发送一个 POST 请求到 `http://127.0.0.1:25001/v1/chat/completions`，提交一个包含对话信息的 JSON 数据，例如用户的问题或指令。
2. **处理请求并生成回复**：服务器接收到请求后，使用加载好的模型和分词器对用户输入的内容进行处理，通过生成函数生成模型的回答，并将回答格式化为规定的 OpenAI API 格式。
3. **返回结果**：服务器将生成的回答以 JSON 格式返回给用户，用户可以通过客户端接收并查看模型生成的文本内容。

先运行下面的代码连接服务端口

In [ ]:
!python ./evalscope-eval/api_model/url.py 

再开启一个新的terminal运行下面的代码

In [ ]:
!bash ./evalscope-eval/api_model/eval_api_eval.sh

最终结果如下

![](./examples/notebook_result.png)

---
## 3、模型推理性能压测

**评估模型在高负载下的表现**
- 识别性能瓶颈：通过模拟高并发请求场景，可以发现模型在处理大量请求时可能出现的性能瓶颈，如响应时间过长、资源占用过高、吞吐量不足等问题。例如，当并发数逐渐增加时，观察模型的响应时间是否呈线性增长，从而判断模型在高负载下的处理能力。
- 测试系统稳定性：确保模型在长时间、高负载运行下不会出现崩溃、异常中断或性能急剧下降的情况，从而评估模型服务的可靠性和稳定性，为模型在实际生产环境中的稳定运行提供保障。

**优化模型性能**
- 指导资源分配：根据压测结果，了解模型在不同并发级别下的资源需求，如 CPU、内存、显存等的使用情况，从而合理分配资源，提高资源利用率，避免资源浪费或不足。
- 调整模型参数：发现模型在推理过程中的不足之处，如某些参数设置不合理导致性能不佳，进而对模型的超参数进行调整和优化，以提升模型的推理效率。

**比较不同模型的性能**
- 选择最优模型：在多个模型中选择最适合特定应用场景的模型。通过在相同的压测条件下对不同模型进行测试，比较它们的性能指标，如响应时间、吞吐量、资源消耗等，从而为实际应用选择性能最优的模型。
- 评估模型改进效果：对于经过优化或改进的模型版本，通过与原始模型进行对比压测，验证改进措施是否有效提升了模型的推理性能，从而为模型的持续优化提供依据。
为模型部署提供依据
- 确定部署规模：根据模型在不同并发请求下的性能表现，预估实际应用中需要部署的模型实例数量、服务器规模等，以满足用户对模型服务的性能需求。
制定应急预案：了解模型在极端情况下的表现，提前制定应对策略，如在流量高峰时的负载均衡策略、资源扩展方案等，以确保模型服务在各种情况下都能正常运行。

In [ ]:
!bash evalscope-eval/swanlab/perf.sh

压测结果可以查看链接👉[SwanLab](https://swanlab.cn/@LiXinYu/perf_benchmark/runs/k0flil25zyxgt1097asz8/chart)